A draculab network is a collection of interconnected units, optionally interacting with a plant. 

Units are continuous-time one-dimensional dynamical systems that implement models of firing-rate
neurons. Connections between units have a temporal delay and a synapse type. Temporal delays are multiples of a minimum delay. Synapses can too be dynamical systems, modifying their weights in response to presynaptic and postsynaptic events.

Plants are continuous dynamical systems that can be modeled with ordinary differential equations. Like units, they can receive input connections from multiple units, and unlike units they can produce several output values, since they can have any finite dimensionality. Plants can be used to model a physical system that is being controlled by the units in the network.



### Draculab brings advantages when the following factors are involved:

* Firing rate units with transmission delays.
* Experimental types of units and synapses, with frequent modifications happening.
* Closed loop experiments where neural controllers interact with a simulated physical plant.

### Main ideas for the architecture:

1. Connection delays are handled in a way inspired by NEST's use of a minimum delay. Draculab also has a minimum delay, called `min_delay`. Dynamics are advanced in discrete time steps of `min_delay` length using SciPy's ODE integrator, obtaining the state of all unit and plant dynamic variables of the network for different time points, and saving this in a buffer. When a past value of a variable is requested, it is obtaned through interpolation on the stored values. 
2. We assume that the activity of units and the state variables of plants change much faster than the state of synapses. Synapses can thus advance their dynamics with user-defined methods that update the synapse's state variables once per simulation step, as opposed to using SciPy's adaptive integrator to obtain values for many time points within the step. This can make the simulation much faster in networks with a large number of dynamic synapses. Moreover, creating new synapse models is straightforward.
3. Connectivity is described using 3 lists: `delays, act, syns`. Separate track of the delays are sometimes kept in the synapses, but in terms of delay steps.
4. All the values required to update synapses are computed in the postsynaptic and presynaptic units, which avoids performing the same computation many times. To do this, the `pre_syn_update` function is called at each update, before the synapses are updated. It obtains any values the synapses need in order to calculate the change in their weights.

Basically, all that `network.run` does is to repeatedly call `unit.update` and `plant.update`, which advances time by the minimal delay.

### Design decisions:

* There is the 'core' draculab (synapses, plants, units, network, draculab, topology), and there is the functionality based on Python (e.g. ei_network).
* Units of time in the network and physical units in the plant are up to the user, and it is up to the user to ensure consistency.
* Naming conventions are simple. All names consist of lowercase words or abbreviations connected by single underscores.
* Units are uniquely identified by a number, called their identifier, or 'id'. The unit identifiers range from 0 to the number of units minus one, being assigned in the order that units were created.
* Documentation is largely contained within the code itself. Python's Docstring conventions are used to document all methods, and abundant comments try to clarify the source code. **Draculab's source code is meant to be understood by the user.** The line between user and developer is meant to be thin.

### Tutorial
0. Installation (python, cython, jupyter)
1. Introduction
1. (hello world) Create 10 sigmoidal units. Create 1 source unit.
   Assign a cosine to the source unit. Connect source to
   sigmoidal. Simulate. Plot outputs.
   Show network.connect documentation help(network.connect) and assign variations.
   Variations include: one input per sigmoidal (to use the one-to-one
   option), a network where each unit receives connections from 2 randomly
   chosen units.
2. Extracting principal components.
3. Controlling a pendulum.
4. The topology module
4. A network that represents inputs (ei_net)
3. ( develper's tutorial) Creating a new unit type.
4. Creating a new synapse type.

TODO: 
* Move docstring to Google format, consider pylint (https://github.com/google/styleguide/blob/gh-pages/pyguide.md#38-comments-and-docstrings)
* Example of custom unit
* Example of user-defined synapse model 
* Example of synapse model using synaptic requirement
* Time comparison agains XPP, Simulink (e.g. against generic dynamical systems simulation software)